In [83]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

<font size=6><b>Lec10.NLP 언어모델
* NLP (Natural Language Processing, 자연어 처리)   * 언어 모델(Language Model, LM)은 언어라는 현상을 모델링하고자 단어 시퀀스(문장)에 확률을 할당(assign)하는 모델 

- ref : https://wikidocs.net/24603

<font size=6><b>언어모델
* N-gram
* <b>빈도기반 
    * <b>BoW(Bag of Words)
    * <b>TF-IDF(Term Frequency - Inverse Document Frequency)
* 코사인 유사도(Cosine Similarity)

# N-gram
 - 입력한 문자열을 N개의 기준 단위로 잘라서 분석하는 방법

* N : 2

In [16]:
# === len으로 표현 === 
text = 'Hello'
for i in range(len(text) - 1):             # 2-gram이므로 문자열의 끝에서 한 글자 앞까지만 반복함
    print(text[i], text[i + 1], sep='')    # 현재 문자와 그다음 문자 출력  
print('-----')


# === zip으로 표현 ===   
text = 'hello'
two_gram = zip(text, text[1:])
for i in two_gram:
    print(i[0], i[1], sep='')

He
el
ll
lo
-----
he
el
ll
lo


In [14]:
text = 'this is python script'
words = text.split()                 # 공백을 기준으로 문자열을 분리하여 리스트로 만듦
 
for i in range(len(words) - 1):      # 2-gram이므로 리스트의 마지막에서 요소 한 개 앞까지만 반복함
    print(words[i], words[i + 1]) 

this is
is python
python script


----

## 정렬 sorted()


In [2]:
my_dict = {'c': 3, 'a': 1, 'b': 2, 'e': 1, 'd': 2}

sorted_dict = sorted(my_dict.items(), reverse = False) # 오름차순
reverse_sorted_dict = sorted(my_dict.items(), reverse = True) # 내림차순

print('keys 오름차순 \n',sorted_dict,'\n ',dict(sorted_dict))
print('keys 내림차순 \n',reverse_sorted_dict,' \n ',dict(reverse_sorted_dict))

keys 오름차순 
 [('a', 1), ('b', 2), ('c', 3), ('d', 2), ('e', 1)] 
  {'a': 1, 'b': 2, 'c': 3, 'd': 2, 'e': 1}
keys 내림차순 
 [('e', 1), ('d', 2), ('c', 3), ('b', 2), ('a', 1)]  
  {'e': 1, 'd': 2, 'c': 3, 'b': 2, 'a': 1}


In [3]:
my_dict = {'code': 3, 'arima': 1, 'banana': 2, 'elephant': 1, 'dimmension': 2}

sorted_dict = sorted(my_dict.items(), key = len)

print('keys 길이(len)오름차순 \n',sorted_dict,'\n ',dict(sorted_dict))

keys 길이(len)오름차순 
 [('code', 3), ('arima', 1), ('banana', 2), ('elephant', 1), ('dimmension', 2)] 
  {'code': 3, 'arima': 1, 'banana': 2, 'elephant': 1, 'dimmension': 2}


In [4]:
my_dict = {'code': 3, 'arima': 1, 'banana': 2, 'elephant': 1, 'dimmension': 2}

sorted_dict = sorted(my_dict.items(), key=lambda x :  x[1] )
reverse_sorted_dict = sorted(my_dict.items(),key=lambda x :  x[1] ,reverse = True)

print('values 기준 오름차순 \n',sorted_dict,'\n ',dict(sorted_dict))
print('values 기준 내림차순 \n',reverse_sorted_dict,' \n ',dict(reverse_sorted_dict))

values 기준 오름차순 
 [('arima', 1), ('elephant', 1), ('banana', 2), ('dimmension', 2), ('code', 3)] 
  {'arima': 1, 'elephant': 1, 'banana': 2, 'dimmension': 2, 'code': 3}
values 기준 내림차순 
 [('code', 3), ('banana', 2), ('dimmension', 2), ('arima', 1), ('elephant', 1)]  
  {'code': 3, 'banana': 2, 'dimmension': 2, 'arima': 1, 'elephant': 1}


---

# BoW(Bag of Words)
* 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
- one-hot 인코딩의 단점인 희소행렬의 단점을 해소시켜줌 

* dictionary items()

In [4]:
mydic= {"no":111, "name":"홍길동"}
print(  mydic.items() )
print(  mydic.keys() )
print(  mydic.values() )

dict_items([('no', 111), ('name', '홍길동')])
dict_keys(['no', 'name'])
dict_values([111, '홍길동'])


In [5]:
from nltk.corpus import stopwords
stop_words_list = stopwords.words('english')
print('불용어 개수 :', len(stop_words_list))
print('불용어 10개 출력 :',stop_words_list[:10])

불용어 개수 : 179
불용어 10개 출력 : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


## CountVectorizer() + 불용어

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop_words_list = stopwords.words('english')
my_words_list = ["because", "I"]

stop_words_list  = stop_words_list + my_words_list

corpus = ['you know I want your love. because I love you.']
# vector = CountVectorizer(stop_words=["because", "I"])  # 사용자정의
# vector = CountVectorizer(stop_words="english")         # 기본
vector = CountVectorizer(stop_words=stop_words_list)     # 기본+사용자정의


# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray()) 

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)

vocab_sorted = sorted(vector.vocabulary_.items(), key = lambda x:x[1], reverse = False)
print('values 기준 오름차순 정렬',vocab_sorted)

keys_sorted = sorted(vector.vocabulary_.items(), reverse = False)
print('keys 알파벳 기준 정렬',keys_sorted)

bag of words vector : [[1 2 1]]
vocabulary : {'know': 0, 'want': 2, 'love': 1}
values 기준 오름차순 정렬 [('know', 0), ('love', 1), ('want', 2)]
keys 알파벳 기준 정렬 [('know', 0), ('love', 1), ('want', 2)]


# DTM
문서 단어 행렬(DTM, Document-Term Matrix)

<img src = 'https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcB2TKv%2Fbtrhsf7YUZh%2FIJ6QYKsmv3Wm53hienhY3k%2Fimg.png'>

* 각 문서에서 등장한 단어의 빈도를 행렬의 값으로 표기해 서로 다른 문서들을 비교
* 다수의 BoW : 각 문서에 대한 BoW를 하나의 행렬로 만든 것

* <단점>
    - 희소 표현(Sparse representation) : 단어 집합의 크기 == 벡터의 차원이 되고 대부분의 값이 0이 된다
    - 단순 빈도 수 기반 접근 : '이다'와 같은 중요하지 않은 최빈도 단어로 문서를 연관지으면??
    - <font color='red'> TF-IDF 필요 (DTM에 불용어와 중요한 단어에 대해서 가중치)</font>

## [실습]

## coo(밀집행렬) -> CSR(희소행렬)
- COO
<pre>
  (0, 0)	3
  (0, 2)	1
  (1, 1)	2
- CSR
<pre>
 [[3 0 1]
 [0 2 0]]

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
corpus = ['먹고 싶은 사과', 
          '먹고 싶은 바나나', 
          '길고 노란 바나나 바나나',
          '저는 과일이 좋아요']

vector = CountVectorizer()    
dtm = pd.DataFrame(vector.fit_transform(corpus).toarray() ,  #coo(밀집행렬) -> CSR(희소행렬)
                  columns = sorted(vector.vocabulary_.keys()),
                 index = ['문장1','문장2','문장3','문장4'])
print(vector.fit_transform(corpus))             # coo(밀집행렬)
print(vector.fit_transform(corpus).toarray())   # CSR(희소행렬)
print(sorted(vector.vocabulary_.keys()))
dtm

  (0, 3)	1
  (0, 6)	1
  (0, 5)	1
  (1, 3)	1
  (1, 6)	1
  (1, 4)	1
  (2, 4)	2
  (2, 1)	1
  (2, 2)	1
  (3, 7)	1
  (3, 0)	1
  (3, 8)	1
[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문장1,0,0,0,1,0,1,1,0,0
문장2,0,0,0,1,1,0,1,0,0
문장3,0,1,1,0,2,0,0,0,0
문장4,1,0,0,0,0,0,0,1,1


# TF_IDF
* 단어 빈도-역 문서 빈도(TF-IDF, Term Frequency-Inverse Document Frequency)
* 우선 DTM을 만든 후, TF-IDF 가중치를 부여
* 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 활용
* TF-IDF = TF * IDF

* 문서(d), 단어(t), 문서총개수(n)
* <font color='red'> $ tf(d,t) $ : 특정 문서 d에서의 특정 단어 t의 등장 횟수</font>
* <font color='red'>  $ df(t) $ : 특정 단어 t가 등장한 문서의 수 </font>
* <font color='red'>  $ idf(d, t) = log (\frac{n}{1+df(t)})$ : df(t) 역수</font>

## [실습]
- ref : https://wikidocs.net/31698

### 공통함수

In [42]:
corpus_list = ['먹고 싶은 사과', 
          '먹고 싶은 바나나', 
          '길고 노란 바나나 바나나',
          '저는 과일이 좋아요']
vocab = ['과일이', '길고', '노란', '먹고', 
         '바나나', '사과', '싶은', '저는', '좋아요']

N = len(corpus_list) # -> 문장의 갯수

# == tf : 문장(d) 내에 단어 (t)의 출현 빈도를 출력
# DTM = Bows... # t: 단어, d : 문장
def tf(t, d): 
    return d.count(t)

# === 문서들(docs)에서 해당 단어(t)의 빈도수
def idf(t):
    df = 0
    for doc in corpus_list:
        df += t in doc
    return np.log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

### TF (DTM = BoWs...)
- 특정 문서 d에서의 특정 단어 t의 등장 횟수

In [25]:
matrix_list = []
for d in corpus_list:
    d_list = [] 
    for t in vocab:
        d_list.append(tf(t, d))
    matrix_list.append(d_list)

matrix_list        

[[0, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 2, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 1]]

### IDF
- (특정 단어 t가 등장한 문서의 수)의 역수

In [43]:
idf_list = []
for t in vocab:
    idf_list.append(idf(t))
idf_ = pd.DataFrame(idf_list, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


### TF-IDF

In [41]:
result = []
for m in corpus_list:
    d_list = [] 
    for v in vocab:
        d_list.append(tfidf(v,m))
    result.append(d_list)
 
tf_idf_df = pd.DataFrame(result, columns = vocab)
tf_idf_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.00000,0.00000,0.00000,0.124939,0.000000,0.30103,0.124939,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.124939,0.124939,0.00000,0.124939,0.00000,0.00000
2,0.00000,0.30103,0.30103,0.000000,0.249877,0.00000,0.000000,0.00000,0.00000
3,0.30103,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.30103,0.30103


In [34]:
tf_arr = np.array(matrix_list)
idf_arr = np.array(idf_list)
tfidf_arr = tf_arr * idf_arr

tf_idf_df = pd.DataFrame(tfidf_arr, columns = vocab)
tf_idf_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


## [실습]

### sklearn.CountVectorizer()

In [87]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_list = ['먹고 싶은 사과',
               '먹고 싶은 바나나',
               '길고 노란 바나나 바나나',
               '저는 과일이 좋아요']
tfidfv = CountVectorizer()

tfidfv_arr = tfidfv.fit_transform(corpus_list).toarray()
voca = tfidfv.vocabulary_
df = pd.DataFrame(tfidfv_arr, columns = sorted(voca))
df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


### sklearn. TfidfVectorizer()
* from sklearn.feature_extraction.text import TfidfVectorizer
* (IDF의 로그항의 분자에 1을 더해주며, 로그항에 1을 더해주고, 
    <b>TF-IDF에 L2 정규화라는 방법으로 값을 조정</b>

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus_list = ['먹고 싶은 사과',
               '먹고 싶은 바나나',
               '길고 노란 바나나 바나나',
               '저는 과일이 좋아요']
tfidfv = TfidfVectorizer()

tfidfv_arr = tfidfv.fit_transform(corpus_list).toarray()
voca = tfidfv.vocabulary_
df2 = pd.DataFrame(tfidfv_arr, columns = sorted(voca))
df2

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
2,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
3,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


# 코사인 유사도(Cosine Similarity)
* 벡터의 크기가 아니라 <b>방향(패턴)</b>에 초점
- 두 문장간의 유사도를 나타냄 값이 1에 가까울수록 유사도가 높음

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbpLIxd%2Fbtq03F9AfHS%2FRTzzEkvHWvqQ5XfQh6rkY1%2Fimg.png"><br>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbayZ5E%2Fbtq04KWSnlM%2FAkYrOmeom4jOCLv2fBf1HK%2Fimg.png">

In [75]:
import numpy as np
from numpy.linalg import norm

A = df.loc[0].values
B = df.loc[1].values
res1 = np.dot(A,B)
res2 = norm(A)*norm(B)
cos_sim = res1/res2
print(f'코사인 유사도 {cos_sim:.5f}')

코사인 유사도 0.66667


## 공통함수

In [94]:
from numpy.linalg import norm 

# np.dot(A,B) : A와 B의 내적 
# norm(A) : A의 크기 -> ||A||
def my_cos_sim(A,B):
    cos_sim = np.dot(A, B) / (norm(A) * norm(B))
    return cos_sim   
    
# sklearn...vectorize.. [1,2,3,]
doc1 = [0,1,2,1]
doc2 = [1,0,1,1]
doc3 = [2,0,2,2]
print(my_cos_sim(doc1,doc2))
print(my_cos_sim(doc1,doc3))
print(my_cos_sim(doc2,doc3))    

0.7071067811865476
0.7071067811865476
1.0000000000000002


## 사용예

In [53]:
text_list = ["어제 비가 와서 날씨가 추워졌다.",
             "나는 고양이를 좋아한다.",
             "나는 강아지를 좋아한다."]

In [59]:
t = list(s.split() for s in text_list)
t

[['어제', '비가', '와서', '날씨가', '추워졌다.'],
 ['나는', '고양이를', '좋아한다.'],
 ['나는', '강아지를', '좋아한다.']]

### Tokenizer()

In [99]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

text_list = ["어제 비가 와서 날씨가 추워졌다.",
             "나는 고양이를 좋아한다.",
             "나는 강아지를 좋아한다."]

token = Tokenizer()
token.fit_on_texts(text_list)

text_seq_list = token.texts_to_sequences(text_list)
vocab = token.word_index

max_length = max(len(sent.split()) for sent in text_list )
padding_res = pad_sequences(sequences=text_seq_list,  
                   maxlen=max_length,padding="pre" )

cs1 = my_cos_sim(padding_res[0], padding_res[1])
cs2 = my_cos_sim(padding_res[0], padding_res[2])
cs3 = my_cos_sim(padding_res[1], padding_res[2])
print(cs1, cs2, cs3)

0.6941980239122336 0.6774960231074738 0.9995786102379159


### CountVectorizer()

In [97]:
cv = CountVectorizer()
cv_arr = cv.fit_transform(text_list).toarray()

for i in range(len(tfidf_arr)):
    for j in range(len(tfidf_arr)):
        if i < j:
            유사도 = my_cos_sim(cv_arr[i],cv_arr[j])
            print(f'문장{i+1}과 문장{j+1}의 유사도 : {유사도}')

문장1과 문장2의 유사도 : 0.0
문장1과 문장3의 유사도 : 0.0
문장2과 문장3의 유사도 : 0.6666666666666667


### TfidfVectorizer()

In [98]:
tfidf_ = TfidfVectorizer()
tfidf_arr = tfidf_.fit_transform(text_list).toarray()

for i in range(len(tfidf_arr)):
    for j in range(len(tfidf_arr)):
        if i < j:
            유사도 = my_cos_sim(tfidf_arr[i],tfidf_arr[j])
            print(f'문장{i+1}과 문장{j+1}의 유사도 : {유사도}')

문장1과 문장2의 유사도 : 0.0
문장1과 문장3의 유사도 : 0.0
문장2과 문장3의 유사도 : 0.5363499141045837


## [실습] 추천 시스템
- ref : https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
- 영화 리뷰

In [110]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
# stop_words_list = stopwords.words('english')
# print('불용어 개수 :', len(stop_words_list))
# print('불용어 10개 출력 :',stop_words_list[:10])


In [106]:
data = pd.read_csv('./datasets/movies_metadata.csv')[:20000]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  20000 non-null  object 
 1   belongs_to_collection  2399 non-null   object 
 2   budget                 20000 non-null  object 
 3   genres                 20000 non-null  object 
 4   homepage               3055 non-null   object 
 5   id                     20000 non-null  object 
 6   imdb_id                19993 non-null  object 
 7   original_language      19999 non-null  object 
 8   original_title         20000 non-null  object 
 9   overview               19865 non-null  object 
 10  popularity             19998 non-null  object 
 11  poster_path            19907 non-null  object 
 12  production_companies   19999 non-null  object 
 13  production_countries   19999 non-null  object 
 14  release_date           19983 non-null  object 
 15  re

C:\Users\Jisu\AppData\Local\Temp\ipykernel_16080\417297325.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./datasets/movies_metadata.csv')[:20000]


In [104]:
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


### 결측처리
- overview

In [107]:
data['overview'].fillna('',inplace=True)

In [108]:
data['overview'].isna().sum()

0

## TF-IDF를 이용해 코사인 유사도 계산
* cosine_sim

In [31]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

# print(tfidf_matrix[0].toarray()) # [[0. 0. 0. ... 0. 0. 0.]]
# print(tfidf.vocabulary_)         # {'led': 24361, 'woody': 46617, 'andy': 2051 ... }

TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [32]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [33]:
data["title"][:5]

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

In [34]:
cos_df = pd.DataFrame(cosine_sim, columns=data["title"], index=data["title"])
cos_df.head()

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,The American President,Dracula: Dead and Loving It,Balto,Nixon,Cutthroat Island,Casino,Sense and Sensibility,Four Rooms,Ace Ventura: When Nature Calls,Money Train,Get Shorty,Copycat,Assassins,Powder,Leaving Las Vegas,Othello,Now and Then,Persuasion,The City of Lost Children,Shanghai Triad,Dangerous Minds,Twelve Monkeys,Wings of Courage,Babe,Carrington,Dead Man Walking,Across the Sea of Time,It Takes Two,Clueless,"Cry, the Beloved Country",Richard III,Dead Presidents,Restoration,Mortal Kombat,To Die For,How To Make An American Quilt,Se7en,Pocahontas,When Night Is Falling,The Usual Suspects,...,Kiss and Tell,"Oui, mais...",Space Adventure Cobra,L'aventure c'est l'aventure,A Season for Miracles,The Awful Truth,1911,Looking for Jackie,The Shaolin Temple,Shaolin Temple 2: Kids from Shaolin,Carol Channing: Larger Than Life,Memory,Martial Arts of Shaolin,Snowmageddon,Stand Up and Fight,Jazz,Barbara,A Liar's Autobiography: The Untrue Story of Monty Python's Graham Chapman,Little Miss Broadway,Miss Annie Rooney,Mr. Belvedere Goes to College,The Hobbit: An Unexpected Journey,Mon Paradis - Der Winterpalast,No Flesh Shall Be Spared,I am Von Höfler Variation on Werther,Born to Defend,Dragon Fight,The Master,Kung Fu Cult Master,The Fitzgerald Family Christmas,Hyde Park on Hudson,Lay the Favorite,Audition/Talent Competition,All's Faire in Love,Crooked Arrows,The Enforcer,Dr. Wai in the Scriptures with No Words,Once Upon a Time in China and America,Contract Killer,The Executor,Never Say... Never!,Calmos,How to Make Love to a Woman,"After Fall, Winter",Violeta Went to Heaven,Rebellion,Versailles,Two in the Wave,Lotte Reiniger: Homage to the Inventor of the Silhouette Film,"RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'"
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.015757,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.041140,0.000000,0.0,0.009941,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.019831,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.006348,0.000000,0.0,0.009321,0.00000,0.000000,0.000000,0.013822,...,0.000000,0.0,0.011365,0.0,0.000000,0.0,0.011791,0.014688,0.013539,0.007355,0.011161,0.000000,0.01452,0.000000,0.000000,0.0,0.000000,0.000000,0.019853,0.027106,0.00000,0.000000,0.0,0.0,0.000000,0.00000,0.117982,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.009979,0.060997,0.000000,0.0,0.0,0.024585,0.0,0.0,0.0
Jumanji,0.015757,1.000000,0.049073,0.0,0.000000,0.051830,0.00000,0.000000,0.106355,0.0,0.007616,0.0,0.0,0.008233,0.0,0.0,0.000000,0.028753,0.000000,0.0,0.000000,0.0,0.006043,0.000000,0.0,0.024285,0.024941,0.006489,0.0,0.000000,0.000000,0.047250,0.0,0.005560,0.004326,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.025758,0.0,0.000000,0.01848,0.008127,0.000000,0.016429,...,0.000000,0.0,0.004063,0.0,0.000000,0.0,0.006281,0.000000,0.000000,0.000000,0.030391,0.000000,0.00000,0.027595,0.043131,0.0,0.011296,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.007418,0.01348,0.009529,0.000000,0.0,0.060063,0.0,0.0,0.000000,0.013191,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.029242,0.000000,0.000000,0.004271,0.0,0.0,0.000000,0.0,0.0,0.0
Grumpier Old Men,0.000000,0.049073,1.000000,0.0,0.025005,0.000000,0.00000,0.006500,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.018349,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.011910,0.0,0.010392,0.000000,0.006408,0.0,0.006683,0.000000,0.000000,0.005282,0.0,0.0,0.0,0.0,0.000000,0.000000,0.007480,0.0,0.000000,0.00000,0.016500,0.016821,0.000000,...,0.000000,0.0,0.000000,0.0,0.007415,0.0,0.000000,0.000000,0.000000,0.015443,0.000000,0.007561,0.00000,0.031215,0.000000,0.0,0.

## 유사영화 Top10 검색
* 영화제목을 넣으면 코사인유사도 Top10 영화 제목을 리턴한다.

In [35]:
cos_df.loc['The Dark Knight Rises'].sort_values(ascending=False)[1:11]

title
The Dark Knight                            0.321521
Batman Forever                             0.315328
Batman Returns                             0.305010
Batman: Under the Red Hood                 0.296967
Batman                                     0.276030
Batman Beyond: Return of the Joker         0.236289
Batman: Year One                           0.209219
Batman: The Dark Knight Returns, Part 1    0.206999
Batman: Mask of the Phantasm               0.196292
Batman Begins                              0.179930
Name: The Dark Knight Rises, dtype: float64